In [1]:
import pandas as pd
import glob
import pickle
import re

In [128]:
# 첫번째 csv 파일에서 티커 가지고 오기
df = pd.read_csv('../Russell_index/russell_20060929.csv', skiprows = 10)
df = df.drop(df.index[-1])
list1 = df['Ticker'].tolist()

# 모든 csv파일에서 겹치는 티커 찾기
path = r'../Russell_index'
all_files = glob.glob(path + '/*csv')
for filename in all_files:
    temp_df = pd.read_csv(filename, skiprows = 10)
    temp_list = temp_df['Ticker'].tolist()
    list1 = [itm for itm in list1 if itm in temp_list]

# 리스트를 DF으로 바꾸기
intersect_df = pd.DataFrame(data = list1)
intersect_df.columns = ['Ticker']

# 피클 파일 열기
with open('../EDGAR_index/ticker_cik.pkl', 'rb') as f:
    Tick_CIK = pickle.load(f)

# intersect_df랑 Tick_CIK 겹치는 부분만 찾기
intersect_df_withCIK = pd.merge(intersect_df, Tick_CIK, how = 'inner', left_on = 'Ticker', right_on = 'Ticker')
intersect_df_withCIK['CIK']=intersect_df_withCIK['CIK'].astype(int)


# Edgar 파일 한개로 합치기
path = r'../EDGAR_index/EDGAR_index'
all_files = glob.glob(path + '/*csv')
dfs = []
for filename in all_files:
    dfs.append(pd.read_csv(filename))
big_frame = pd.concat(dfs, ignore_index = True)
big_frame['CIK'] = big_frame['CIK'].astype(int)

# intersect_df_withCIK랑 Edgar 한개로 합친 파일이랑 겹치는 부분만 찾기 + 10-K & 10-Q 만 찾기
CIK_Ticker_Edgar = pd.merge(intersect_df_withCIK, big_frame, how = 'inner', left_on = 'CIK', right_on = 'CIK')
CIK_Ticker_Edgar = CIK_Ticker_Edgar[(CIK_Ticker_Edgar['Form Type'] == '10-K') | (CIK_Ticker_Edgar['Form Type'] == '10-Q')]

# File name 바꾸기
f = lambda x: ''.join(re.findall('\d+.*', x)[0].split('.')[0].split('-'))
CIK_Ticker_Edgar['File Name'] = CIK_Ticker_Edgar['File Name'].apply(f)
g = lambda x: 'https://www.sec.gov/Archives/edgar/data/'+str(x)+'Financial_Report.xlsx'
CIK_Ticker_Edgar['File Name'] = CIK_Ticker_Edgar['File Name'].apply(g)
CIK_Ticker_Edgar = CIK_Ticker_Edgar[['CIK','Ticker','Form Type','File Name']]

# CIK, Ticker로 Index
CIK_Ticker_Edgar = CIK_Ticker_Edgar.set_index(['CIK','Ticker'])

Form Type                                          File Name
CIK   Ticker                                                             
72162 NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-K  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
      NL          10-Q  https://www.sec.gov/Archives/edgar/data/72162/...
...                ...                                                ...
34088 XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-K  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-K  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-K  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-K  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-Q  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM         10-K  https://www.sec.gov/Archives/edgar/data/34088/...
      XOM        

In [133]:
CIK_Ticker_Edgar.to_csv('Edgar_changed_Filename.csv', encoding = 'utf-8')

In [123]:
# df = pd.read_csv('../Russell_index/russell_20060929.csv', skiprows = 10)
# df = df.drop(df.index[-1])
# list1 = df['Ticker'].tolist()
# set1 = set(list1)
# #print(list1[-1])
# #print(len(list1))

# path = r'../Russell_index'
# all_files = glob.glob(path + '/*csv')

# for filename in all_files:
#     temp_df = pd.read_csv(filename, skiprows = 10)
#     temp_list = temp_df['Ticker'].tolist()
#     temp_set = set(temp_list)
#     set1 = set1 & temp_set

# list2 = list(set1)
# intersect_df1 = pd.DataFrame(data = list2)
# intersect_df1.columns = ['Ticker']
# intersect_df1

# intersect_df_withCIK1 = pd.merge(intersect_df1, Tick_CIK, how = 'inner', left_on = 'Ticker', right_on = 'Ticker')
# intersect_df_withCIK1

In [ ]:
# 첫번째 csv 파일에서 티커 가지고 오기
df = pd.read_csv('../Russell_index/russell_20060929.csv', skiprows = 10)
df = df.drop(df.index[-1])
list1 = df['Ticker'].tolist()

# 모든 csv파일에서 겹치는 티커 찾기
path = r'../Russell_index'
all_files = glob.glob(path + '/*csv')

for filename in all_files:
    temp_df = pd.read_csv(filename, skiprows = 10)
    temp_list = temp_df['Ticker'].tolist()
    list1 = [itm for itm in list1 if itm in temp_list]

# 리스트를 DF으로 바꾸기
intersect_df = pd.DataFrame(data = list1)
intersect_df.columns = ['Ticker']

# 피클 파일 열기
with open('../EDGAR_index/ticker_cik.pkl', 'rb') as f:
    Tick_CIK = pickle.load(f)

# intersect_df랑 Tick_CIK 겹치는 부분만 찾기
intersect_df_withCIK = pd.merge(intersect_df, Tick_CIK, how = 'inner', left_on = 'Ticker', right_on = 'Ticker')
    
intersect_df_withCIK
